In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=16, app_name='west0_userseg', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 16 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_userseg...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:25


In [2]:
post_df = mysql.read_table(spark, "labs", "workshop_user_post_seg_tmp")

In [3]:
post_start_date = "2022-07-13"
post_end_date = "2022-09-06"

In [7]:
def get_workshop_rate(start_date, end_date):
    bp_purchase = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where((col("Currency").like("%bp%")) & (col("AnalyticEventType") == "workshop"))
    gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name").like("%workshop%")).withColumnRenamed("account_id", "AccountId")
    open_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date)
    craft = load_schema.lobby(spark, "pc", "live", "WorkshopCrafted", start_date, end_date)
    disassemble = load_schema.lobby(spark, "pc", "live", "WorkshopDisassembled", start_date, end_date)
    repurpose = load_schema.lobby(spark, "pc", "live", "WorkshopRepurposed", start_date, end_date)
    special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where(col("Currency") == "artisanstoken")
    
    workshop_user = (open_crate.groupBy("AccountId").agg(sum("OpenAmount").alias("amount")).withColumn("action", lit("open"))) \
        .unionByName(craft.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("craft"))) \
        .unionByName(disassemble.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("disassemble"))) \
        .unionByName(repurpose.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("repurpose"))) \
        .unionByName(special_craft.groupBy("AccountId").agg(sum("Amount").alias("amount")).withColumn("action", lit("special_craft"))) \
        .unionByName(gcoin.groupBy("AccountId").agg(sum("qty").alias("amount")).withColumn("action", lit("gcoin"))) \
        .unionByName(bp_purchase.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("bp_workshop")))
    
    return workshop_user

In [8]:
workshop_use = get_workshop_rate(post_start_date, post_end_date)

In [9]:
post_df_user = post_df.withColumnRenamed("account_id", "AccountId").select("AccountId", "post_seg", "post_group")
post_user_workshop_use = workshop_use.join(post_df_user, "AccountId", "right")

In [11]:
mysql.insert_table(post_user_workshop_use, "labs", "post_user_workshop_use")